In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'
import gc

In [0]:
path = ''
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

In [0]:
train_df = pd.read_csv(path+"train2.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])

In [0]:
test_df = pd.read_csv(path+"test2.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])

In [0]:
len_train = len(train_df)

In [0]:
train_df=train_df.append(test_df)

In [0]:
del test_df; gc.collect()

524

In [0]:
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
train_df['wday']  = pd.to_datetime(train_df.click_time).dt.dayofweek.astype('uint8')

In [0]:
train_df.head()

,app,channel,click_id,click_time,device,ip,is_attributed,os,hour,day,wday
0,3,379,NaN,2017-11-06 14:32:21,1,83230,0.0,13,14,6,0
1,3,379,NaN,2017-11-06 14:33:34,1,17357,0.0,19,14,6,0
2,3,379,NaN,2017-11-06 14:34:12,1,35810,0.0,13,14,6,0
3,14,478,NaN,2017-11-06 14:34:52,1,45745,0.0,13,14,6,0
4,3,379,NaN,2017-11-06 14:35:08,1,161007,0.0,13,14,6,0


In [0]:
gp = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'qty'})


In [0]:
train_df = train_df.merge(gp, on=['ip','day','hour'], how='left')

In [0]:
del gp; gc.collect()

14

In [0]:
train_df.head()

,app,channel,click_id,click_time,device,ip,is_attributed,os,hour,day,wday,qty
0,3,379,NaN,2017-11-06 14:32:21,1,83230,0.0,13,14,6,0,1
1,3,379,NaN,2017-11-06 14:33:34,1,17357,0.0,19,14,6,0,1
2,3,379,NaN,2017-11-06 14:34:12,1,35810,0.0,13,14,6,0,1
3,14,478,NaN,2017-11-06 14:34:52,1,45745,0.0,13,14,6,0,1
4,3,379,NaN,2017-11-06 14:35:08,1,161007,0.0,13,14,6,0,1


In [0]:
gp = train_df[['ip','app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})

In [0]:
train_df = train_df.merge(gp, on=['ip','app'], how='left')

In [0]:
del gp; gc.collect()

82

In [0]:
gp = train_df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})

In [0]:
train_df = train_df.merge(gp, on=['ip','app', 'os'], how='left')
del gp; gc.collect()

63

In [0]:
train_df['qty'] = train_df['qty'].astype('uint16')
train_df['ip_app_count'] = train_df['ip_app_count'].astype('uint16')
train_df['ip_app_os_count'] = train_df['ip_app_os_count'].astype('uint16')

In [0]:
train_df.head()

,app,channel,click_id,click_time,device,ip,is_attributed,os,hour,day,wday,qty,ip_app_count,ip_app_os_count
0,3,379,NaN,2017-11-06 14:32:21,1,83230,0.0,13,14,6,0,1,33,17
1,3,379,NaN,2017-11-06 14:33:34,1,17357,0.0,19,14,6,0,1,35,9
2,3,379,NaN,2017-11-06 14:34:12,1,35810,0.0,13,14,6,0,1,26,8
3,14,478,NaN,2017-11-06 14:34:52,1,45745,0.0,13,14,6,0,1,122,38
4,3,379,NaN,2017-11-06 14:35:08,1,161007,0.0,13,14,6,0,1,1,1


In [0]:
train_df.describe()

,app,channel,click_id,device,ip,is_attributed,os,hour,day,wday,qty,ip_app_count,ip_app_os_count
count,1.999998e+06,1.999998e+06,999999.000000,1.999998e+06,1.999998e+06,999999.000000,1.999998e+06,1.999998e+06,1.999998e+06,1.999998e+06,1.999998e+06,1.999998e+06,1.999998e+06
mean,1.213342e+01,2.567658e+02,499999.000000,1.417431e+01,7.507942e+04,0.001693,2.080998e+01,9.999734e+00,8.000000e+00,2.000000e+00,2.390587e+02,4.449922e+01,9.026737e+00
std,1.374344e+01,1.314087e+02,288674.990256,1.953356e+02,4.855316e+04,0.041111,3.992068e+01,5.999762e+00,2.000001e+00,2.000001e+00,7.462763e+02,1.726325e+02,3.163954e+01
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,3.000000e+00,0.000000,0.000000e+00,4.000000e+00,6.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,3.000000e+00,1.350000e+02,249999.500000,1.000000e+00,3.688900e+04,0.000000,1.300000e+01,4.000000e+00,6.000000e+00,0.000000e+00,2.500000e+01,3.000000e+00,1.000000e+00
50%,1.200000e+01,2.370000e+02,499999.000000,1.000000e+00,7.227100e+04,0.000000,1.800000e+01,9.000000e+00,8.000000e+00,2.000000e+00,5.000000e+01,8.000000e+00,2.000000e+00
75%,1.500000e+01,3.790000e+02,749998.500000,1.000000e+00,1.062230e+05,0.000000,1.900000e+01,1.600000e+01,1.000000e+01,4.000000e+00,1.130000e+02,2.200000e+01,6.000000e+00
max,6.450000e+02,4.980000e+02,999998.000000,3.379000e+03,2.127730e+05,1.000000,7.150000e+02,1.600000e+01,1.000000e+01,4.000000e+00,6.437000e+03,2.330000e+03,6.730000e+02


In [0]:
from sklearn.preprocessing import LabelEncoder
train_df[['app','device','os', 'channel', 'hour', 'day', 'wday']].apply(LabelEncoder().fit_transform)

,app,device,os,channel,hour,day,wday
0,3,1,13,110,1,0,0
1,3,1,19,110,1,0,0
2,3,1,13,110,1,0,0
3,14,1,13,153,1,0,0
4,3,1,13,110,1,0,0
5,3,1,16,110,1,0,0
6,3,1,23,110,1,0,0
7,3,1,19,110,1,0,0
8,3,1,13,110,1,0,0
9,63,1,22,145,1,0,0


In [0]:
test_df = train_df[len_train:]
train_df = train_df[:len_train]

In [0]:
y_train = train_df['is_attributed'].values

In [0]:
train_df.drop(['click_id', 'click_time','ip','is_attributed'],1,inplace=True)

In [0]:
train_df.head()

,app,channel,device,os,hour,day,wday,qty,ip_app_count,ip_app_os_count
0,3,379,1,13,14,6,0,1,33,17
1,3,379,1,19,14,6,0,1,35,9
2,3,379,1,13,14,6,0,1,26,8
3,14,478,1,13,14,6,0,1,122,38
4,3,379,1,13,14,6,0,1,1,1


In [0]:
import tensorflow as tf

In [0]:
from tensorflow.python.keras.layers import Input, Embedding, Dense, Flatten, Dropout, concatenate
from tensorflow.python.keras.layers import BatchNormalization, SpatialDropout1D
from tensorflow.python.keras.callbacks import Callback
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.optimizers import Adam

In [0]:
max_app = np.max([train_df['app'].max(), test_df['app'].max()])+1
max_ch = np.max([train_df['channel'].max(), test_df['channel'].max()])+1
max_dev = np.max([train_df['device'].max(), test_df['device'].max()])+1
max_os = np.max([train_df['os'].max(), test_df['os'].max()])+1
max_h = np.max([train_df['hour'].max(), test_df['hour'].max()])+1
max_d = np.max([train_df['day'].max(), test_df['day'].max()])+1
max_wd = np.max([train_df['wday'].max(), test_df['wday'].max()])+1
max_qty = np.max([train_df['qty'].max(), test_df['qty'].max()])+1
max_c1 = np.max([train_df['ip_app_count'].max(), test_df['ip_app_count'].max()])+1
max_c2 = np.max([train_df['ip_app_os_count'].max(), test_df['ip_app_os_count'].max()])+1

In [0]:
def get_keras_data(dataset):
    X = {
        'app': np.array(dataset.app),
        'ch': np.array(dataset.channel),
        'dev': np.array(dataset.device),
        'os': np.array(dataset.os),
        'h': np.array(dataset.hour),
        'd': np.array(dataset.day),
        'wd': np.array(dataset.wday),
        'qty': np.array(dataset.qty),
        'c1': np.array(dataset.ip_app_count),
        'c2': np.array(dataset.ip_app_os_count)
    }
    return X
train_df = get_keras_data(train_df)

In [0]:
emb_n = 50
dense_n = 1000

In [0]:
in_app = Input(shape=[1], name = 'app')
emb_app = Embedding(max_app, emb_n)(in_app)
in_ch = Input(shape=[1], name = 'ch')
emb_ch = Embedding(max_ch, emb_n)(in_ch)
in_dev = Input(shape=[1], name = 'dev')
emb_dev = Embedding(max_dev, emb_n)(in_dev)
in_os = Input(shape=[1], name = 'os')
emb_os = Embedding(max_os, emb_n)(in_os)
in_h = Input(shape=[1], name = 'h')
emb_h = Embedding(max_h, emb_n)(in_h) 
in_d = Input(shape=[1], name = 'd')
emb_d = Embedding(max_d, emb_n)(in_d) 
in_wd = Input(shape=[1], name = 'wd')
emb_wd = Embedding(max_wd, emb_n)(in_wd) 
in_qty = Input(shape=[1], name = 'qty')
emb_qty = Embedding(max_qty, emb_n)(in_qty) 
in_c1 = Input(shape=[1], name = 'c1')
emb_c1 = Embedding(max_c1, emb_n)(in_c1) 
in_c2 = Input(shape=[1], name = 'c2')
emb_c2 = Embedding(max_c2, emb_n)(in_c2) 
fe = concatenate([(emb_app), (emb_ch), (emb_dev), (emb_os), (emb_h), 
                 (emb_d), (emb_wd), (emb_qty), (emb_c1), (emb_c2)])

In [0]:
s_dout = SpatialDropout1D(0.2)(fe)
x = Flatten()(s_dout)
x = Dropout(0.2)(Dense(dense_n,activation='relu')(x))
x = Dropout(0.2)(Dense(dense_n,activation='relu')(x))
outp = Dense(1,activation='sigmoid')(x)
model = Model(inputs=[in_app,in_ch,in_dev,in_os,in_h,in_d,in_wd,in_qty,in_c1,in_c2], outputs=outp)

In [0]:
batch_size = 20000
epochs = 2
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(train_df) / batch_size) * epochs
lr_init, lr_fin = 0.001, 0.0001
lr_decay = exp_decay(lr_init, lr_fin, steps)
optimizer_adam = Adam(lr=0.001, decay=lr_decay)
model.compile(loss='binary_crossentropy',optimizer=optimizer_adam,metrics=['accuracy'])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
app (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
ch (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
dev (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
os (InputLayer)                 [(None, 1)]          0                                            
____________________________________________________________________________________________

In [0]:
model.fit(train_df, y_train, batch_size=batch_size, epochs=2, shuffle=True, verbose=2)
model.save_weights('dl_support.h5')

Epoch 1/2
999999/999999 - 8s - loss: 0.0557 - acc: 0.9917
Epoch 2/2
999999/999999 - 8s - loss: 0.0067 - acc: 0.9983


In [0]:
sub = pd.DataFrame()
sub['click_id'] = test_df['click_id'].astype('int')
y_test = test_df['is_attributed'].values
test_df.drop(['click_id', 'click_time','ip','is_attributed'],1,inplace=True)
test_df = get_keras_data(test_df)

In [0]:
sub['is_attributed'] = model.predict(test_df, batch_size=batch_size, verbose=2)
print("writing....")
sub.to_csv('dl_support.csv',index=False)

999999/999999 - 3s
writing....


In [0]:
test_df


{'app': array([ 9,  9, 21, ...,  6, 13, 18], dtype=uint16),
 'c1': array([ 2, 26,  7, ...,  1,  1, 19], dtype=uint16),
 'c2': array([1, 3, 5, ..., 1, 1, 1], dtype=uint16),
 'ch': array([107, 466, 128, ..., 125, 477, 439], dtype=uint16),
 'd': array([10, 10, 10, ..., 10, 10, 10], dtype=uint8),
 'dev': array([1, 1, 1, ..., 1, 1, 1], dtype=uint16),
 'h': array([4, 4, 4, ..., 4, 4, 4], dtype=uint8),
 'os': array([ 3,  3, 19, ..., 13, 18, 78], dtype=uint16),
 'qty': array([ 12, 135,  76, ...,  11,  18,  86], dtype=uint16),
 'wd': array([4, 4, 4, ..., 4, 4, 4], dtype=uint8)}

In [0]:
train_df

{'app': array([ 3,  3,  3, ..., 12, 18, 12], dtype=uint16),
 'c1': array([33, 35, 26, ...,  3,  5,  5], dtype=uint16),
 'c2': array([17,  9,  8, ...,  1,  1,  2], dtype=uint16),
 'ch': array([379, 379, 379, ..., 178, 107, 265], dtype=uint16),
 'd': array([6, 6, 6, ..., 6, 6, 6], dtype=uint8),
 'dev': array([1, 1, 1, ..., 1, 1, 1], dtype=uint16),
 'h': array([14, 14, 14, ..., 16, 16, 16], dtype=uint8),
 'os': array([13, 19, 13, ..., 15, 42, 13], dtype=uint16),
 'qty': array([ 1,  1,  1, ..., 31, 36, 60], dtype=uint16),
 'wd': array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)}